In [10]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler,scale
# import cPickle

In [38]:
TRAINING_RATE = 0.5
TESTING_RATE = 1 - TRAINING_RATE
MISSING_RATE = 0.8
# QUERY_RATE = 0.2
np.random.seed(0)

In [56]:
data = pd.read_csv("../dat/wine_quality/winequality-white.csv",sep=';')

In [57]:
def preprocessing(d):
    #Add normalization code here if necessary
#     d.ix[:,0:-1] = scale(d.ix[:,0:-1])
    #0-1 scale
    min_max_scaler = MinMaxScaler()
    d.ix[:,0:-1] = min_max_scaler.fit_transform(d.ix[:,0:-1])
    d['quality'] = d['quality'].apply(lambda x: int(x) -3)
    d['fil'] = d['quality'].apply(lambda x:0 if x==0 or x==5 or x==6 else 1)
    d = d[d['fil']==1]
    d.drop('fil',axis=1,inplace=True)
    d['quality'] = d['quality'].apply(lambda x: 1 if x==3 or x == 4 else 0)
    d = d.iloc[np.random.permutation(len(d))]
    t = int(len(d) * TRAINING_RATE)
    tn_data = d.iloc[0:t,:]
    tt_data = d.iloc[t:,:]
    
    tn_X = tn_data.ix[:,0:-1]
    tn_Y = tn_data.ix[:,-1]
    tt_X = tt_data.ix[:,0:-1]
    tt_Y = tt_data.ix[:,-1]
    
    return tn_X,tn_Y, tt_X, tt_Y
#     return tt_X, tt_Y

In [14]:
def TestMissingGenerate(tt_data,mr=MISSING_RATE):
    missing_entry = []
    row , col= range(tt_data.shape[0]),range(tt_data.shape[1])
    
    while len(missing_entry) < tt_data.shape[0] * tt_data.shape[1] * mr:
        r = np.random.choice(row)
        c = np.random.choice(col)
        
        if (r,c) not  in missing_entry:
            missing_entry.append((r,c))
        else:
            continue
    
    for me in missing_entry:
        tt_data.set_value(me[0],tt_data.columns[me[1]],np.nan)
    return tt_data, missing_entry

In [15]:
def Imputation(d,missing):
    d = d.fillna(0.0)
    model = NMF(n_components=4, init='random', random_state=0)
    model.fit(d)
    H = model.components_
    W = model.fit_transform(d)
    d_prime = W.dot(H)
    
#     d = d.values
#     for m in missing:
#         d[m[0]][m[1]] = d_prime[m[0]][m[1]]
#         #d[m[0]][m[1]] = 0.0
    return d_prime

In [58]:
tn_X,tn_Y,tt_X,tt_Y = preprocessing(data)
# tt_X,tt_Y = preprocessing(data)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
tn_X = tn_X.reset_index(drop=True)
tn_Y = tn_Y.reset_index(drop=True)
tt_X = tt_X.reset_index(drop=True)
tt_oracle = tt_X.copy()

In [74]:
data.ix[range(20)]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,fil
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,3,1
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,3,1
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,3,1
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,3,1
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,3,1
5,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,3,1
6,0.230769,0.235294,0.096386,0.098160,0.106825,0.097561,0.294664,0.150183,0.418182,0.290698,0.258065,3,1
7,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,3,1
8,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,3,1
9,0.413462,0.137255,0.259036,0.013804,0.103858,0.090592,0.278422,0.128976,0.454545,0.267442,0.483871,3,1


In [79]:
M = LogisticRegression()

In [82]:
M.fit(tn_X, tn_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [84]:
a = M.predict(tt_X)

In [87]:
np.sum(a == tt_Y) / tt_Y.count()

0.73478075776926355

In [83]:
tt_oracle.to_csv("../dat/wine_quality/wine_zeroone_oracle.csv",index=False,header=None)

In [84]:
tt_X, missing_entry = TestMissingGenerate(tt_X)

In [85]:
tt_X = tt_X.fillna(0)

In [86]:
tt_X.to_csv("../dat/wine_quality/wine_zeroone.csv",index=False,header=None)

In [87]:
tt_imp = Imputation(tt_X,missing_entry)

In [88]:
def calRMSE(result, oracle):
    return np.sum((result - oracle.values) ** 2)

In [89]:
calRMSE(tt_imp,tt_oracle)

1650.3400123807216

In [ ]:
#CASE1. NON_SCALE
#MF         #AF
16932450.98 5173595.76  #MISSING_RATE = 0.3
44081856.77 13897144.15 #MISSING_RATE = 0.8

#CALE2. [0,1]SCALE
#MF     #AF
578.20  309.47 #MISSING_RATE = 0.3
1650.34 333.09 #MISSING_RATE = 0.8
